In [218]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
#models
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split ##
from sklearn.metrics import classification_report,confusion_matrix
from nltk import pos_tag
from nltk.corpus import wordnet
import nltk

# stop_words = set(stopwords.words('english'))


df = pd.read_csv("spam.csv",encoding='latin-1')
df = df[['v1','v2']]
df.rename(columns={'v1':'class','v2':'msg'},inplace=True)
df.head()

,class,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [187]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger_eng')

In [188]:
df2 = pd.read_csv("spam_ds2.csv")
df2.drop(columns=['URL','EMAIL','PHONE'],axis = 1, inplace=True)
df2.rename(columns={'LABEL':'class','TEXT':'msg'},inplace=True)
df2.head()


,class,msg
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,ham,What's up? Do you want me to come online? If y...
2,ham,So u workin overtime nigpun?
3,ham,"Also sir, i sent you an email about how to log..."
4,Smishing,Please Stay At Home. To encourage the notion o...


In [189]:
df = pd.concat([df,df2],axis=0,ignore_index=True)
df.head()


,class,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [190]:
df.tail()

,class,msg
11538,ham,:( but your not here....
11539,ham,Becoz its &lt;#&gt; jan whn al the post ofic...
11540,ham,Its a valentine game. . . send dis msg to all ...
11541,ham,We r outside already.
11542,ham,The Xmas story is peace.. The Xmas msg is love...


In [191]:
df3 = pd.read_csv('ham_call_messages.csv')
df3 = df3[['class','msg']]
df3.head()

,class,msg
0,ham,"Hey, call me when you're free."
1,ham,I'll call you after lunch.
2,ham,Call me once you reach home.
3,ham,Can you call mom later?
4,ham,Don’t forget to call me in the evening.


In [192]:
df = pd.concat([df,df3],axis=0,ignore_index=True)
df.tail()

,class,msg
11687,spam,"Hey, it's me! Time-sensitive info, act fast. C..."
11688,spam,"Guess what? It’s amazing, trust me! Call me on..."
11689,spam,"Hey, it's me! Check your rewards here. Call me..."
11690,spam,"Hey dude, You’ve got to check this now. Call m..."
11691,spam,Hi there! You’ve got to check this now. Call m...


In [193]:
lemmatizer = WordNetLemmatizer()


def get_pos_tag(pos):
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

def preprocess_msg(msg):
    msg = msg.lower()
    msg = ''.join([c for c in msg if c not in string.punctuation])
    word_list = word_tokenize(msg)
    # word_list = [w for w in word_list if w not in stop_words]
    word_pos_tags = pos_tag(word_list)
    word_list = [lemmatizer.lemmatize(w,get_pos_tag(p)) for w,p in word_pos_tags]
    processed_msg = ' '.join(word_list)
    
    return processed_msg


In [194]:

df["no_of_chars"] = df['msg'].apply(len)
df = df[df['no_of_chars']<450]
df = df.drop(columns=['no_of_chars'],axis=1)
df['cleaned_msg'] = df['msg'].apply(preprocess_msg)
df.head()

,class,msg,cleaned_msg
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he go to usf he live around h...


In [195]:
df['class'] = df['class'].map({'ham':0,'spam':1})
df.head()


,class,msg,cleaned_msg
0,0.0,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,0.0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1.0,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,0.0,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,0.0,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he go to usf he live around h...


In [196]:
df.drop(columns=['msg'],axis=1, inplace=True)
df.head()

,class,cleaned_msg
0,0.0,go until jurong point crazy available only in ...
1,0.0,ok lar joking wif u oni
2,1.0,free entry in 2 a wkly comp to win fa cup fina...
3,0.0,u dun say so early hor u c already then say
4,0.0,nah i dont think he go to usf he live around h...


In [197]:
print(df['class'].value_counts())

class
0.0    9729
1.0    1283
Name: count, dtype: int64


In [198]:
tf_idf = TfidfVectorizer(stop_words=None)
tf_idf.fit(df['cleaned_msg']) #fitting on entire data of messages

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [199]:
#for multinomial Naive bayes
ham_mnb = df[df['class']==0].sample(n=3000,random_state=7)
spam = df[df['class']==1] #spam is same for all models
df_mnb = pd.concat([ham_mnb,spam]).sample(frac=1,random_state=7).reset_index(drop=True)
df_mnb.head()

,class,cleaned_msg
0,0.0,sbut he have some luck2 catch put down
1,0.0,no im in the same boat still here at my mom ch...
2,0.0,do not b late love mum
3,0.0,i want to see your pretty pussy
4,1.0,boltblue tone for 150p reply poly or mono eg p...


In [200]:
df_mnb['class'].value_counts()

class
0.0    3000
1.0    1283
Name: count, dtype: int64

In [201]:

X = tf_idf.transform(df_mnb['cleaned_msg'])
y = df_mnb['class']
# X_train,X_test ,y_train,y_test  = train_test_split(X,y,test_size=0.2,random_state=7) ##

mnb = MultinomialNB()
# mnb.fit(X_train,y_train) ##
mnb.fit(X,y) #

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [202]:
# y_test_pred = mnb.predict(X_test) ##
# print("confusion matrix:\n",confusion_matrix(y_test,y_test_pred))##
# print("classification report:\n",classification_report(y_test,y_test_pred))##

In [214]:
#for randomforest classifier
ham_rf = df[df['class']==0].sample(n=6000,random_state=8)
df_rf = pd.concat([ham_rf,spam]).sample(frac=1,random_state=10).reset_index(drop=True)
X = tf_idf.transform(df_rf['cleaned_msg'])
y = df_rf['class']
rf = RandomForestClassifier(n_estimators=150,max_depth=None, random_state=7,class_weight={0:1,1:10})
rf.fit(X,y)

,n_estimators,150
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [204]:
# y_pred = rf.predict(X)
# print("confusion matrix\n",confusion_matrix(y,y_pred))
# print("classification report:\n",classification_report(y,y_pred))

In [ ]:
#for svc
ham_svc = df[df['class']==0].sample(n=4000,random_state=12)
df_svc = pd.concat([ham_rf,spam]).sample(frac=1,random_state=10).reset_index(drop=True)
X = tf_idf.transform(df_svc['cleaned_msg'])
y = df_svc['class']
svc = SVC(kernel='rbf',class_weight={0:1,1:4},probability=True)
svc.fit(X,y)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,True
,tol,0.001
,cache_size,200
,class_weight,"{0: 1, 1: 4}"
,verbose,False


In [206]:
# y_pred = svc.predict(X)
# print("confusion matrix\n",confusion_matrix(y,y_pred))
# print("classification report:\n",classification_report(y,y_pred))

In [ ]:
def predict_msg(input_msg):
    input_msg = preprocess_msg(input_msg)
    input_msg_v = tf_idf.transform([input_msg])
    models = [mnb,rf,svc]
    op_list = []
    probabs = []
    for model in models:
        op_list.append(model.predict(input_msg_v)[0])
        probabs.append(model.predict_proba(input_msg_v)[0])
    print("Three model predictions:",op_list)
    output = 0 if op_list.count(0)>op_list.count(1) else 1
    # print(output)
    #print(op_list)
    ham_probab = np.mean([x[0] for x in probabs])
    # print(f"ham probability:{ham_probab:.3f}")
    ham_percent = ham_probab*100
    print(f"ham percentage: {ham_percent:.2f}%")
    print(f"spam percentage: {(100-ham_percent):.2f}%")
    return 'HAM' if output==0 else 'SPAM' if output == 1 else 'ERROR Occured'

print(predict_msg("hi hello ra madhav , what are you doing, call me,there is a free buffet grab now"))
print(predict_msg("hii, ra madhav  how are you dooing ,call me when u"))
print(predict_msg("call me when u free ra"))

Three model predictions: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
ham percentage: 88.10%
spam percentage: 11.90%
HAM
Three model predictions: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
ham percentage: 94.45%
spam percentage: 5.55%
HAM
Three model predictions: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
ham percentage: 88.00%
spam percentage: 12.00%
HAM


In [240]:
print(predict_msg("Congratulations! You've won ₹10,000 cash prize. Call now to claim your reward. Limited time offer!"))
print(predict_msg("Hey, it’s me! You won’t believe what I just found,it is free.,call me on this number 9823XXXXXX 😮"))

Three model predictions: [np.float64(1.0), np.float64(1.0), np.float64(1.0)]
ham percentage: 5.37%
SPAM
Three model predictions: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
ham percentage: 61.13%
HAM


In [217]:
import joblib
joblib.dump(mnb,'model_mnb.pkl')
joblib.dump(rf,'model_rf.pkl')
joblib.dump(svc,'model_svc.pkl')
joblib.dump(tf_idf,'vectorizer.pkl')

['vectorizer.pkl']